# OpenMic 语音合成质量评估 (MOS) Demo

本 Notebook 演示如何根据 **ITU-T P.800 标准** 对脱口秀语音合成效果进行主观评估 (Subjective Evaluation)。
我们特别关注以下维度：**停顿合理性**、**重音准确性**、**情感表达度**。

**注意：由于还并未进行问卷调查，现在只有假的数据，后续需要导入真实问卷数据分析**

---

## 1. 评估方法说明 (Methodology)

### 评估标准：ACR (Absolute Category Rating)
采用绝对类别评分法，使用 5 分制量表（Likert Scale）。

| 分数 | 描述 (Description) | 详细定义 |
| :---: | :--- | :--- |
| **5** | **Excellent (优)** | 自然流畅，极具感染力，完全像真人表演。 |
| **4** | **Good (良)** | 听感舒适，有少量不明显的机械感，但不影响幽默感传达。 |
| **3** | **Fair (中)** | 能听出是合成语音，节奏尚可，但缺乏表演张力。 |
| **2** | **Poor (差)** | 明显的机械感，停顿或重音错误，干扰了对笑点的理解。 |
| **1** | **Bad (劣)** | 无法忍受，断句混乱，完全没有脱口秀的感觉。 |

## 2. 问卷设计 (Questionnaire Design)

针对每一段生成的脱口秀音频（Sample），评估员需要从以下四个维度进行打分：

### 维度定义

1.  **总体自然度 (Overall Naturalness)**
    *   *Question*: 这段语音听起来像真实的脱口秀演员在表演吗？
    *   *关注点*: 音质、连贯性、不仅是朗读更要是"表演"。

2.  **停顿合理性 (Pause Appropriateness)**
    *   *Question*: 停顿的位置和时长是否恰当？
    *   *关注点*: **气口**是否准确？包袱（Punchline）抛出前是否有期待性的停顿？笑点后是否留有足够的反应时间？

3.  **重音准确性 (Emphasis Accuracy)**
    *   *Question*: 是否正确强调了关键信息？
    *   *关注点*: 逻辑重音是否突出了段子中的对比、转折或夸张部分？

4.  **情感表达度 (Emotion Expressiveness)**
    *   *Question*: 语气是否符合文本的情境？
    *   *关注点*: 吐槽时的犀利、自嘲时的无奈、模仿他人时的语气切换是否到位？

---

## 3. 模拟数据生成 (Data Simulation)

假设我们要评估 5 个不同版本的音频片段 (`Clip_A` 到 `Clip_E`)。
邀请了 20 位评估员 (`User_01` 到 `User_20`)。

*下面我们编写代码来生成一份模拟的评测数据。*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置随机种子以保证复现性
np.random.seed(2026)

# 定义样本参数
num_raters = 20
samples = ['Clip_A_Baseline', 'Clip_B_NoFillers', 'Clip_C_WithRhythm', 'Clip_D_FullFeatures', 'Clip_E_HumanRecord']
dimensions = ['Naturalness', 'Pause', 'Emphasis', 'Emotion']

data = []

# 模拟打分逻辑
for rater_id in range(1, num_raters + 1):
    rater_name = f"User_{rater_id:02d}"
    
    for sample in samples:
        for dim in dimensions:
            # 设定预期的基础分数分布 (模拟不同模型的表现差异)
            if sample == 'Clip_A_Baseline':        # 基线模型：表现平平
                base = 3.0
            elif sample == 'Clip_B_NoFillers':     # 无语气词：自然度偏低
                base = 2.5 if dim == 'Naturalness' else 3.0
            elif sample == 'Clip_C_WithRhythm':    # 加入节奏控制：停顿得分高
                base = 4.0 if dim == 'Pause' else 3.5
            elif sample == 'Clip_D_FullFeatures':  # 完整版 (OpenMic)：各项较高
                base = 4.2
            elif sample == 'Clip_E_HumanRecord':   # 真人录音：接近满分
                base = 4.8
            
            # 添加随机扰动 (模拟主观差异)
            score = base + np.random.normal(0, 0.6)
            
            # 截断在 1-5 之间并取整
            final_score = int(np.clip(round(score), 1, 5))
            
            data.append({
                'rater': rater_name,
                'sample': sample,
                'dimension': dim,
                'score': final_score
            })

# 转换为 DataFrame
df = pd.DataFrame(data)

# 展示数据结构
print(f"Total ratings collected: {len(df)}")
display(df.sample(5))

## 4. 数据处理与分析 (Data Processing)

我们需要计算每个样本在每个维度上的**平均主观得分 (MOS)** 以及 **95% 置信区间 (95% Confidence Interval)**。

公式：
$$ CI_{95} = 1.96 \times \frac{\sigma}{\sqrt{N}} $$

其中 $\sigma$ 是标准差，$N$ 是样本量（评分人数）。

In [ ]:
def calculate_mos_stats(df):
    # 按样本和维度分组计算 Mean, Std, Count
    stats = df.groupby(['sample', 'dimension'])['score'].agg(['mean', 'std', 'count']).reset_index()
    
    # 计算 95% 置信区间
    stats['ci'] = 1.96 * (stats['std'] / np.sqrt(stats['count']))
    
    # 格式化显示列
    stats['mos_display'] = stats.apply(lambda x: f"{x['mean']:.2f} ± {x['ci']:.2f}", axis=1)
    
    return stats

mos_results = calculate_mos_stats(df)

# 展示计算结果
print("MOS 统计结果预览 (Top 10):")
# 重新排序以便观察: 按 Sample 分组
display(mos_results.sort_values(by=['sample', 'dimension']).head(8))

## 5. 结果可视化 (Visualization)

通过图表直观地对比不同模型（Clip）在立项书要求的三个关键维度（停顿、重音、情感）上的表现。

In [ ]:
# 设置绘图风格
sns.set_theme(style="whitegrid", context="notebook")

plt.figure(figsize=(14, 7))

# 绘制柱状图
# errorbar=('ci', 95) 会自动计算并绘制置信区间
chart = sns.barplot(
    data=df,
    x="dimension",
    y="score",
    hue="sample",
    errorbar=("ci", 95),
    palette="viridis",
    capsize=0.05,
    gap=0.1
)

# 图表美化
plt.title("MOS Evaluation Results: OpenMic Speech Module", fontsize=16, pad=20)
plt.ylabel("Mean Opinion Score (1-5)", fontsize=12)
plt.xlabel("Evaluation Dimensions", fontsize=12)
plt.legend(title="Audio Samples", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.ylim(1, 5.5)  # 留出顶部空间

# 添加参考线 (Good Quality Threshold)
plt.axhline(y=4.0, color='r', linestyle='--', alpha=0.3, label='Good Quality Threshold')

plt.tight_layout()
plt.show()

## 6. 结论分析 (Conclusion)

从模拟数据的结果图中，我们可以得出以下分析（示例）：

1.  **停顿合理性 (Pause)**: `Clip_C_WithRhythm` 和 `Clip_D_FullFeatures` 明显优于基线 `Clip_A`，证明了 OpenMic 中的 `EmotionRhythmController` 模块在处理包袱节奏上的有效性。
2.  **情感表达 (Emotion)**: 即使是 `Clip_D` (完整版)，在情感表达上与真人 (`Clip_E`) 仍有差距，这提示我们后续可以在 ChatTTS 的情感 Prompt 工程上继续优化。
3.  **总体表现**: 引入表演标记（Markers）后，MOS 分数普遍提升了 0.5-1.0 分，达到了预期目标。

此评估流程将被用于后续的模型迭代验收。